In [14]:
using Revise, Plots, LinearAlgebra, LaTeXStrings, Random, Distributions, BenchmarkTools

In [2]:
includet("../../src/SparseSpikes.jl")
using .SparseSpikes

Define ground truth amplitudes and positions

In [3]:
domain  = [[0, 1], [0, 1]]

2-element Vector{Vector{Int64}}:
 [0, 1]
 [0, 1]

In [4]:
# Define the plot
num_points = 64

plt_grid_x1 = [domain[1][1] + i * (domain[1][2] - domain[1][1]) / num_points for j in 0:num_points, i in 0:num_points]
plt_grid_x2 = [domain[2][1] + j * (domain[2][2] - domain[2][1]) / num_points for j in 0:num_points, i in 0:num_points]

grid = range(0, stop=1, length=(num_points + 1))

0.0:0.015625:1.0

In [5]:
ops = gaussian_operators_2D(0.02, plt_grid_x1, plt_grid_x2)

Operators(var"#ϕ#32"{Int64}(Core.Box(var"#gauss2D#31"{Float64, Vector{Float64}, Vector{Float64}, Matrix{Float64}, Float64}(0.0971404681957369, [0.0, 0.015625, 0.03125, 0.046875, 0.0625, 0.078125, 0.09375, 0.109375, 0.125, 0.140625  …  0.859375, 0.875, 0.890625, 0.90625, 0.921875, 0.9375, 0.953125, 0.96875, 0.984375, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Core.Box(#= circular reference @-2 =#), [0.0 0.0; 0.0 0.015625; … ; 1.0 0.984375; 1.0 1.0], 1250.0)), 4225), var"#Φ#33"{var"#ϕ#32"{Int64}, Int64}(var"#ϕ#32"{Int64}(Core.Box(var"#gauss2D#31"{Float64, Vector{Float64}, Vector{Float64}, Matrix{Float64}, Float64}(0.0971404681957369, [0.0, 0.015625, 0.03125, 0.046875, 0.0625, 0.078125, 0.09375, 0.109375, 0.125, 0.140625  …  0.859375, 0.875, 0.890625, 0.90625, 0.921875, 0.9375, 0.953125, 0.96875, 0.984375, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Core.B

In [428]:
n_spikes = 16

Random.seed!(18)


# Ensure values are within the domain [0, 1] and not too concentrated around the origin
x0 = [rand(n_spikes), rand(n_spikes)]
x0 = [round.(x0[1], digits=3), round.(x0[2], digits=3)]

a0 = clamp.(abs.(round.(rand(Normal(0.9, 0.1), n_spikes), digits=3)), 0, 1)

# x0 = [[0.5], [0.5]]
# a0 = [1.0]

μ0 = DiscreteMeasure(x0, a0)

DiscreteMeasure([[0.188, 0.401, 0.481, 0.203, 0.589, 0.188, 0.817, 0.395, 0.212, 0.372, 0.014, 0.432, 0.004, 0.151, 0.942, 0.757], [0.021, 0.935, 0.23, 0.164, 0.426, 0.443, 0.374, 0.05, 0.801, 0.744, 0.005, 0.627, 0.633, 0.19, 0.931, 0.067]], [0.685, 1.0, 1.0, 0.92, 0.92, 0.691, 0.929, 0.81, 0.728, 1.0, 0.949, 0.81, 0.912, 0.889, 0.868, 0.833], 2, 16)

In [16]:
@benchmark ops.Φ($μ0...)

BenchmarkTools.Trial: 5671 samples with 1 evaluation per sample.
 Range (min … max):  682.200 μs …   7.511 ms  ┊ GC (min … max): 0.00% … 88.18%
 Time  (median):     762.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   876.645 μs ± 378.489 μs  ┊ GC (mean ± σ):  7.54% ± 12.36%

  ▃█▇▅▅▅▅▅▄▃▂▂                                                  ▁
  █████████████████▇▇▆▃▆▆▅▄▅▃▁▁▁▁▁▁▁▁▁▁▃▄▅▅▅▆▆▆▆▇▆▇▆▆▆▆▆▇▆▅▇▇▇▆ █
  682 μs        Histogram: log(frequency) by time       2.48 ms <

 Memory estimate: 1.07 MiB, allocs estimate: 76.

In [8]:
@benchmark ops.ϕ($x0...)

BenchmarkTools.Trial: 6060 samples with 1 evaluation per sample.
 Range (min … max):  674.900 μs …   9.913 ms  ┊ GC (min … max): 0.00% … 91.08%
 Time  (median):     719.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   819.664 μs ± 353.209 μs  ┊ GC (mean ± σ):  7.20% ± 12.16%

  ██▅▃▆▆▄▃▁▁                                                    ▂
  ████████████▇▇█▇▆▄▃▄▅▄▄▁▃▁▅▆▇▇▇▇█▇█▇▇▇▆▆▆▆▆▅▅▅▃▃▁▄▄▄▃▄▅▁▁▄▁▄▅ █
  675 μs        Histogram: log(frequency) by time       2.49 ms <

 Memory estimate: 1.00 MiB, allocs estimate: 66.

In [443]:
y0 = ops.Φ(μ0...)

prob = BLASSO(y0, ops, domain, 0.0047)
t = @timed solve!(prob, :SFW, options=Dict(:maxits => length(a0) + 5))

@info "SFW total = $(t.time) s,   GC = $(t.gctime) s, proportion = $(t.gctime / t.time)"

display(prob.μ)

DiscreteMeasure([[0.21200004402252742, 0.1514802794167257, 0.5890000337509771, 0.01450026901717923, 0.8170000247783918, 0.4010000935570139, 0.48100001676705034, 0.1880001139223154, 0.005842619524891424, 0.4319999097773161, 0.18799977943060922, 0.37200009684808527, 0.39500002090720776, 0.20253577363027525, 0.9419998856586583, 0.7569999902009693], [0.8010000735786708, 0.18975970775554235, 0.426000020437815, 0.006631086267802782, 0.3740000490220451, 0.9349999613287284, 0.22999999087497636, 0.021184443090299886, 0.6330001305001941, 0.6270001623649591, 0.4429999696155879, 0.7439997332091881, 0.04999997500016151, 0.16423224840239387, 0.9310000141279172, 0.06699997315660042]], [0.6312328975087205, 0.799381246218044, 0.8232329066834733, 0.811768808573595, 0.8322329269686468, 0.9032329110651814, 0.9032329148649197, 0.5865243193405427, 0.7775683142210806, 0.713234878257463, 0.5942329214044397, 0.9032348711831993, 0.7132320141227534, 0.8303914071208106, 0.7712328378350731, 0.7362328910189082], 2,

In [444]:
sort(prob.μ.a)

16-element Vector{Float64}:
 0.5865243193405427
 0.5942329214044397
 0.6312328975087205
 0.7132320141227534
 0.713234878257463
 0.7362328910189082
 0.7712328378350731
 0.7775683142210806
 0.799381246218044
 0.811768808573595
 0.8232329066834733
 0.8303914071208106
 0.8322329269686468
 0.9032329110651814
 0.9032329148649197
 0.9032348711831993

In [445]:
y0 = ops.Φ(μ0...)

prob = BLASSO(y0, ops, domain, 0.0047)
t = @timed solve!(prob, :BSFW, options=Dict(:maxits => length(a0) + 30))

display("BSFW total = $(t.time) s,   GC = $(t.gctime) s, proportion = $(t.gctime / t.time)")

display(prob.μ)

"BSFW total = 2.5945122 s,   GC = 0.3182154 s, proportion = 0.12264941363544175"

DiscreteMeasure([[0.20253947672142206, 0.5890004279119966, 0.816999172297872, 0.40100006980640074, 0.48100004163064475, 0.18800002981045363, 0.005842426078148677, 0.43199985835756854, 0.18799998395273948, 0.37200020192826233, 0.3949999703171684, 0.20252788851034892, 0.15148059615351894, 0.9420007085135315, 0.7570000001983642, 0.014500305264632403, 0.21199995297947516], [0.16423910482848506, 0.42600051259681737, 0.3739995302593097, 0.934999959849792, 0.22999972700950047, 0.021185059235003786, 0.6330000635416544, 0.6270002875226391, 0.4429999007490269, 0.743999819788841, 0.05000012772681952, 0.16421784765330433, 0.18976102624489, 0.9309994668173176, 0.06699990108544562, 0.006631268538711278, 0.8010006349205655]], [0.5493005222954819, 0.8232329059626093, 0.8322329256920451, 0.9032329110678651, 0.9032329147957273, 0.5865224425494895, 0.7775704000850183, 0.7132348783239723, 0.5942329211818385, 0.903234870919004, 0.7132320138762828, 0.2810926549003296, 0.7993785352995476, 0.7712328350865478,

In [446]:
sort(prob.μ.a)

17-element Vector{Float64}:
 0.2810926549003296
 0.5493005222954819
 0.5865224425494895
 0.5942329211818385
 0.631504758564545
 0.7132320138762828
 0.7132348783239723
 0.7362328909705996
 0.7712328350865478
 0.7775704000850183
 0.7993785352995476
 0.8117682226213558
 0.8232329059626093
 0.8322329256920451
 0.9032329110678651
 0.9032329147957273
 0.903234870919004

In [440]:
sort(μ0.a)

16-element Vector{Float64}:
 0.685
 0.691
 0.728
 0.81
 0.81
 0.833
 0.868
 0.889
 0.912
 0.92
 0.92
 0.929
 0.949
 1.0
 1.0
 1.0